In [ ]:
import pathlib as pl
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, LineString, Point
import flopy
import flopy.utils.triangle
import flopy.utils.voronoi

In [ ]:
ws = pl.Path('./mf6_radvor')
name = 'radvor'
tempdir = pl.Path(ws, "temp")
tempdir.mkdir(parents=True, exist_ok=True)

In [ ]:
def get_circle(increment, radius, xoff, yoff):
    theta = np.arange(0, 2 * np.pi, increment)
    x = xoff + radius * np.cos(theta)
    y = yoff + radius * np.sin(theta)
    circle_poly = [(x, y) for x, y in zip(x, y)]
    return circle_poly

# we start by creating a polygon (circle_poly), which is a list of
# (x,y) points that define the circle
#theta = np.arange(0.0, 2 * np.pi, 0.1)
h1 = 1 # head on edge
h0 = h1 * 10 # head in middle
radius_outer = 80000.0
xoff = radius_outer
yoff = radius_outer
circle_poly = get_circle(0.01, radius_outer, xoff, yoff)

radius_inner = 5000.0
circle_inner_poly = get_circle(0.1, radius_inner, xoff, yoff)

fig = plt.figure(figsize=(4, 4))
ax = plt.subplot(1, 1, 1, aspect="equal")

x, y = list(zip(*circle_poly))
ax.plot(x, y, "b-")

x, y = list(zip(*circle_inner_poly))
ax.plot(x, y, "r-")

In [ ]:
# We can then use the Triangle class and Triangle program
# to make the mesh, as follows.
maximum_area = 5000**2
tri = flopy.utils.triangle.Triangle(maximum_area=maximum_area, angle=30, model_ws=tempdir)
tri.add_polygon(circle_poly)
tri.add_polygon(circle_inner_poly)
tri.add_hole((xoff, yoff))
tri.add_region((60000, 60000), maximum_area=maximum_area)
tri.build(verbose=False)
# fig = plt.figure(figsize=(5, 5))
# ax = plt.subplot(1, 1, 1, aspect="equal")
#pc = tri.plot(ax=ax)

# And now for the voronoi grid
voronoi_grid = flopy.utils.voronoi.VoronoiGrid(tri)
fig = plt.figure(figsize=(5, 5))
ax = plt.subplot(1, 1, 1, aspect="equal")
voronoi_grid.plot(ax=ax, facecolor="none")

In [ ]:
modelgrid = flopy.discretization.VertexGrid(**voronoi_grid.get_gridprops_vertexgrid(), nlay=1)
gi = flopy.utils.GridIntersect(modelgrid)
lso = LineString([p for p in circle_poly])
edge_cells = np.array(gi.intersects(lso)["cellids"], dtype=int)

lsi = LineString([p for p in circle_inner_poly])
center_cells = np.array(gi.intersects(lsi)["cellids"], dtype=int)

#center_cells = np.array(gi.intersect(Point(xoff, yoff))["cellids"], dtype=int)
a = np.zeros(modelgrid.nnodes, dtype=int)
a[edge_cells] = 1
a[center_cells] = 2

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
pmv.plot_array(a)


In [ ]:
def h_analytical(r, r0, r1, h0, h1):
    rho = (1/r0 - 1/r) / (1/r0 - 1/r1)
    h = ((1 - rho) * h0 ** (13./3.) + rho * h1 ** (13./3.)) ** (3./13.)
    return h

r = np.sqrt((modelgrid.xcellcenters - xoff) ** 2 + (modelgrid.ycellcenters - yoff) ** 2)
r0 = radius_inner
r1 = radius_outer
h_analytical_solution = h_analytical(r, r0, r1, h0, h1)

In [ ]:
exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name=exe_name)
tdis = flopy.mf6.ModflowTdis(sim)

nouter, ninner = 200, 30
hclose, rclose, relax = 1e-8, 1e-8, 0.97
imsswf = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="DBD",
    under_relaxation_theta=0.9,
    under_relaxation_kappa=0.0001,
    under_relaxation_gamma=0.0,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    preconditioner_levels=7,
    # backtracking_number=5,
    # backtracking_tolerance=1.0,
    # backtracking_reduction_factor=0.3,
    # backtracking_residual_limit=100.0,
    filename=f"{name}.ims",
)

swf = flopy.mf6.ModflowOlf(sim, modelname=name, save_flows=True)

# change ncpl to ncells
disv_gridprops = voronoi_grid.get_disv_gridprops()
disv_gridprops["nodes"] = disv_gridprops.pop("ncpl")
disv2d = flopy.mf6.ModflowOlfdisv2D(
    swf, 
    bottom=0.,
    **disv_gridprops,
)

dfw = flopy.mf6.ModflowOlfdfw(
    swf,
    print_flows=False,
    save_flows=True,
    save_velocity=True,
    manningsn=1.0,
    idcxs=None,
)

ic = flopy.mf6.ModflowOlfic(swf, strt=0.305)


chdspd = [[(j,), h1] for j in edge_cells]
chdspd += [[(j,), h0] for j in center_cells]  
chd = flopy.mf6.ModflowOlfchd(swf, stress_period_data=chdspd, pname="CHD-1")

oc = flopy.mf6.ModflowOlfoc(
    swf,
    budget_filerecord=f"{name}.bud",
    stage_filerecord=f"{name}.stage",
    saverecord=[
        ("STAGE", "ALL"),
        ("BUDGET", "ALL"),
    ],
    printrecord=[
        ("BUDGET", "ALL"),
    ],
)


In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
fpth = ws / f"{name}.stage"
stage_obj = flopy.utils.HeadFile(fpth, text="STAGE")
stage = stage_obj.get_data().flatten()

fpth = ws / f"{name}.bud"
cbb = flopy.utils.CellBudgetFile(fpth, precision="double")
velocity = cbb.get_data(text="DATA-VCOMP")[0]

nodes = stage.shape[0]
vx = np.ones((nodes), dtype=float) * 1.0e30
vy = np.ones((nodes), dtype=float) * 1.0e30
vz = np.ones((nodes), dtype=float) * 1.0e30
n0 = velocity["node"] - 1
vx[n0] = velocity["vx"]
vy[n0] = velocity["vy"]
vz[n0] = velocity["vz"]
vx = vx.reshape(nodes)
vy = vy.reshape(nodes)
vz = vz.reshape(nodes)

In [ ]:
# make a color map of fixed colors
from matplotlib import colors
cmap = colors.ListedColormap(['none', 'blue'])
bounds=[0, 1]
norm = colors.BoundaryNorm(bounds, cmap.N)
ibd = np.zeros(swf.dis.nodes.get_data(), dtype=int)
ibd[edge_cells] = 1
ibd[center_cells] = 1
chd_array = np.ma.masked_where((ibd == 0), ibd)

with flopy.plot.styles.USGSPlot():
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_aspect(1.)
    swf = sim.olf[0]
    pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
    qm = pmv.plot_array(chd_array, cmap=cmap, norm=norm)
    pmv.plot_grid(linewidths=0.1)
    ax.set_xlabel("X location, in meters")
    ax.set_ylabel("Y location, in meters")
    fig.savefig("../../doc/figures/oned-voronoi-grid.pdf", dpi=300)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
cb = pmv.plot_array(stage)
plt.colorbar(cb, shrink=0.5)
ax.set_title("Model")

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
cb = pmv.plot_array(h_analytical_solution)
plt.colorbar(cb, shrink=0.5)
ax.set_title("Analytical")

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
residual = stage - h_analytical_solution
cb = pmv.plot_array(residual)
plt.colorbar(cb, shrink=0.5)
ax.set_title("Residual")

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_aspect(1.)
    swf = sim.olf[0]
    pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
    pmv.plot_grid(linewidths=0.1, color="gray")
    hmin = stage.min()
    hmax = stage.max()
    levels = np.linspace(1, 9, 9)
    cs1 = pmv.contour_array(h_analytical_solution, levels=levels, colors="red", linewidths=2)
    cs2 = pmv.contour_array(stage, levels=levels, colors="black", linestyles="dotted", linewidths=2)
    clabels = ax.clabel(cs1, cs1.levels, inline=True, fontsize=10, colors="black")
    [txt.set_bbox(dict(facecolor='white', edgecolor='none', pad=0)) for txt in clabels]
    ax.set_xlabel("X location, in meters")
    ax.set_ylabel("Y location, in meters")
    fig.savefig("../../doc/figures/oned-voronoi-results.pdf", dpi=300)

In [ ]:
h_analytical_solution.min(), h_analytical_solution.max(), stage.min(), stage.max()